# Testy Statysyczne
### Wczytanie danych

In [30]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
songs = pd.read_csv('../data_sets/songs.csv')
songs.columns = ['artysci', 'tytuly', 'data', 'najwyzsza-pozycja', 'liczba-tygodni-w-notowaniu']
display(songs.head())

songs_top_10 = pd.read_csv('../data_sets/songs_top_10.csv')
songs_top_10['duration_ms'] = songs_top_10['duration_ms'].apply(lambda x: x / 1000 / 60)
songs_top_10.rename(columns={'duration_ms': 'duration'}, inplace=True)
songs_top_10.columns = ['artysci','tytul','data', 'akustycznosc', 'tanecznosc', 'czas-trwania-m', 'energia', 'instrumentalnosc', 'wykonanie-na-zywo', 'lirycznosc']
display(songs_top_10.head())

,artysci,tytuly,data,najwyzsza-pozycja,liczba-tygodni-w-notowaniu
0,"""Groove"" Holmes",Misty,1966-09-03,44,11
1,"""Groove"" Holmes",What Now My Love,1966-10-15,96,3
2,"""Little"" Jimmy Dickens",May The Bird Of Paradise Fly Up Your Nose,1965-12-18,15,10
3,"""Pookie"" Hudson",I Know I Know,1963-05-25,96,1
4,"""Weird Al"" Yankovic",Amish Paradise,1996-07-13,53,16


,artysci,tytul,data,akustycznosc,tanecznosc,czas-trwania-m,energia,instrumentalnosc,wykonanie-na-zywo,lirycznosc
0,"""Weird Al"" Yankovic","White & Nerdy (Parody of ""Ridin'"" by Chamillio...",2006-10-21,0.0986,0.791,2.844000,0.613,0.000000,0.0765,0.0763
1,$uicideboy$,"...And to Those I Love, Thanks for Sticking Ar...",2019-03-09,0.1240,0.792,2.808167,0.511,0.000090,0.1400,0.0409
2,'68 Comeback,Peepin' & Hidin' / Baby What You Want Me to Do,1983-03-12,0.0247,0.373,3.904883,0.691,0.024800,0.2120,0.0316
3,'Til Tuesday,Voices Carry,1985-07-13,0.0282,0.583,4.392883,0.574,0.000002,0.1140,0.0239
4,*NSYNC,(God Must Have Spent) A Little More Time On Yo...,1999-03-06,0.4490,0.375,4.024883,0.527,0.000000,0.3050,0.0507



## Wprowadzenie MP3 a liczba tygodni na liście
Na wykresach widać że średnia liczba tygodni notowaniu zwiększyła się wraz z wprowadzeniem MP3. Sprawdzimy czy jest to różnica statystycznie istotna.
### Wybór testu
Mamy dwie próby niezależne, skale przedziałową, musimy sprawdzić czy dane są opisane rozkłądem normalnym. W tym celu użyjemy testu Shapiro-Wilka (próba ma więcej niz 5000 więc test ten może być użyty). Jeśli dane nie są opisane rozkładem normalnym, użyjemy testu nieparametrycznego Mann-Whitneya. Jeśli dane są opisane rozkładem normalnym, użyjemy testu t-Studenta.
### Test Shapiro-Wilka
$H_0$ - dane są opisane rozkładem normalnym <br>
$H_1$ - dane nie są opisane rozkładem normalnym <br>
$\alpha = 0.5$

In [8]:
data = songs['liczba-tygodni-w-notowaniu']
stat, p_value = stats.shapiro(data)
print("Statystyka testowa:", stat)
print("Wartość p-value:", p_value)
alpha = 0.05
if p_value > alpha:
    print("Dane wydają się pochodzić z rozkładu normalnego.")
else:
    print("Dane nie wydają się pochodzić z rozkładu normalnego.")

Statystyka testowa: 0.903465211391449
Wartość p-value: 0.0
Dane nie wydają się pochodzić z rozkładu normalnego.


### Wybór testu c.d.
Z testu Shapiro-Wilka wynika, że dane nie są opisane rozkładem normalnym, więc użyjemy testu nieparametrycznego Mann-Whitneya.
### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - przed mp3(do 2000) <br>
**Grupa 2** - po mp3(od 2000) <br>
$\alpha = 0.5$

In [21]:
songs['data'] = pd.to_datetime(songs['data'])
data1 = songs[songs['data'] < '2000-01-01']
data2 = songs[songs['data'] >= '2000-01-01']
stat, p_value = stats.mannwhitneyu(data1['liczba-tygodni-w-notowaniu'], data2['liczba-tygodni-w-notowaniu'])
print("Statystyka testowa U:", stat)
# pvalue z naukowej notacji do zwykłej
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 91431488.0
Wartość p-value: 0.0000501768
Obserwowane różnice między grupami są istotne.


### Wnioski
P-value jest mniejsze od $\alpha$, więc odrzucamy $H_0$ na rzecz $H_1$. Istnieje istotna różnica w liczbie tygodni na liście przed i po wprowadzeniu mp3. Po wprowadzeniu mp3 liczba tygodni na liście jest większa i może to być spowodowane nowym medium słuchania muzyki. Może to wynikać z zwiększenia dostępności do muzyki co za tym idzie większą liczbą słuchaczy.

## Pojawienie się nowych instrumentów i technologi a muzyka akustycza
Wraz z pojawieniem się nowych instrumentów i technologi muzyka stała się bardziej elektroniczna, zaczeły pojawiać się nowe instrumenty. Według wykresów widać ustępstwo muzyki akustycznej. Sprawdzimy czy jest to różnica statystycznie istotna. Wprowadzenie nowych instrumetów szacuje się na rok 1960/1970.
### Wybór testu
Tak jak poprzednio sprawdzimy czy dane są opisane rozkłądem normalnym (test Shapiro-Wilka). Jeśli dane nie są opisane rozkładem normalnym, użyjemy testu nieparametrycznego Mann-Whitneya. Jeśli dane są opisane rozkładem normalnym, użyjemy testu t-Studenta.
### Test Shapiro-Wilka
$H_0$ - dane są opisane rozkładem normalnym <br>
$H_1$ - dane nie są opisane rozkładem normalnym <br>
$\alpha = 0.5$

In [31]:
data = songs_top_10['akustycznosc']
stat, p_value = stats.shapiro(data)
print("Statystyka testowa:", stat)
print("Wartość p-value:", p_value)
alpha = 0.05
if p_value > alpha:
    print("Dane wydają się pochodzić z rozkładu normalnego.")
else:
    print("Dane nie wydają się pochodzić z rozkładu normalnego.")

Statystyka testowa: 0.8817464113235474
Wartość p-value: 0.0
Dane nie wydają się pochodzić z rozkładu normalnego.


### Wybór testu c.d.
Ponownie dane nie są opisane rozkładem normalnym dlatego wybieramy test nieparametryczny Mann-Whitneya.
### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - przed 1965(przed instrumentami elektrycznymi) <br>
**Grupa 2** - po 1965(po instrumentami elektrycznymi) <br>
$\alpha = 0.5$

In [33]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '1965-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '1965-01-01']
stat, p_value = stats.mannwhitneyu(data1['akustycznosc'], data2['akustycznosc'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 2252126.5
Wartość p-value: 0.0000000000
Obserwowane różnice między grupami są istotne.


### Wnioski
P-value jest mniejsze niż alpha więc odrzucamy $H_0$ na rzecz $H_1$. Istnieje istotna różnica w akustyczności utworów przed nowymi instrumentami oraz po. Po wprowadzeniu nowych instrumentów muzyka stała się bardziej elektroniczna.
### Serwisy Streamingowe a muzyka
Od 2010 roku serwisy streamingowe zaczeły dominować na rynku muzyki. Zmienił sie sposób dystrybucji muzyki oraz zarobku dla artystów. Od tego czasu liczy się przyciągnięcie słuchacza przez conajmniej 30 sekund. Niektórzy twierdzą że spowodowało to coraz więcej muzyki z szybko wchodzącym wokalem oraz dominującym wokalem. Sprawdzmy czy jest to różnica statystycznie istotna na podstawie liryczność, instrumentalności oraz długości trwania utworu.
### Liryczność a serwisy streamingowe
#### Wybór testu
Powtarzamy krok pierwszy z poprzednich testów.
#### Test Shapiro-Wilka
$H_0$ - dane są opisane rozkładem normalnym <br>
$H_1$ - dane nie są opisane rozkładem normalnym <br>
$\alpha = 0.5$

In [34]:
data = songs_top_10['lirycznosc']
stat, p_value = stats.shapiro(data)
print("Statystyka testowa:", stat)
print("Wartość p-value:", p_value)
alpha = 0.05
if p_value > alpha:
    print("Dane wydają się pochodzić z rozkładu normalnego.")
else:
    print("Dane nie wydają się pochodzić z rozkładu normalnego.")

Statystyka testowa: 0.557726263999939
Wartość p-value: 0.0
Dane nie wydają się pochodzić z rozkładu normalnego.


#### Wybór testu c.d.
Dane ponownie nie wydają się pochodzić z testu normalnego. Dlatego wybieramy test nieparametryczny Mann-Whitneya.
#### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - przed 2010 (przed serwisami streamingowymi) <br>
**Grupa 2** - po 2010 (po serwisami streamingowymi) <br>
$\alpha = 0.5$

In [36]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '2010-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '2010-01-01']
stat, p_value = stats.mannwhitneyu(data1['lirycznosc'], data2['lirycznosc'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 785510.5
Wartość p-value: 0.0000000000
Obserwowane różnice między grupami są istotne.


#### Wątpliwości
Według testu są różnice pomiedzy dwoma populacjiami, jednak według wykresów nie ma dużej różnicy w liryczności utworów pomiedzy wprowadzeniem mp3 a serwisami streamingowymi. Sprawdzmy to jeszcze raz ale tym razem w przypadku grupy 2 weźmiemy przedział 2000-2010
#### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - 2000-2010 (przed serwisami streamingowymi, po mp3) <br>
**Grupa 2** - po 2010 (po serwisami streamingowymi) <br>
$\alpha = 0.5$

In [45]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '2010-01-01']
data1 = data1[data1['data'] >= '2000-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '2010-01-01']
stat, p_value = stats.mannwhitneyu(data1['lirycznosc'], data2['lirycznosc'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 184358.5
Wartość p-value: 0.2221349121
Brak istotnych różnic między grupami.


#### Wnioski z drugiego testu
P-value jest większe niż alpha więc nie odrzucamy $H_0$. W takim razie nie ma zmiany pomiedzy wprowadzeniem mp3 a wprowadzeniem serwisów streamingowych. Sprawdzmy czy jest różnica statystyczna miedzy wprowadzeniem mp3 a lirycznością.
#### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - przed 2000 <br>
**Grupa 2** - po 2000 <br>
$\alpha = 0.5$

In [46]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '2000-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '2000-01-01']
stat, p_value = stats.mannwhitneyu(data1['lirycznosc'], data2['lirycznosc'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 1211966.0
Wartość p-value: 0.0000000000
Obserwowane różnice między grupami są istotne.


#### Wnioski po wszystkich testach
Po wprowadzeniu serwisów streamingowych liryczność utworów się nie zmieniła, proces ten zaczął pojawiać się już po wprowadzeniu mp3.
### Instrumentalność a serwisy streamingowe
Ponownie porównamy dane z roku 2000-2010 z danymi po 2010. Jest to bardziej wiarygodne.
#### Wybór testu
Powtarzamy krok pierwszy z poprzednich testów.
#### Test Shapiro-Wilka
$H_0$ - dane są opisane rozkładem normalnym <br>
$H_1$ - dane nie są opisane rozkładem normalnym <br>
$\alpha = 0.5$

In [47]:
data = songs_top_10['instrumentalnosc']
stat, p_value = stats.shapiro(data)
print("Statystyka testowa:", stat)
print("Wartość p-value:", p_value)
alpha = 0.05
if p_value > alpha:
    print("Dane wydają się pochodzić z rozkładu normalnego.")
else:
    print("Dane nie wydają się pochodzić z rozkładu normalnego.")

Statystyka testowa: 0.2561796307563782
Wartość p-value: 0.0
Dane nie wydają się pochodzić z rozkładu normalnego.


#### Wybór testu c.d.
Dane ponownie nie pochodzą z rozkłądu normalnego wybieramy zatem test nieparametryczny Mann-Whitneya.
#### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - 2000 - 2010 (przed serwisami streamingowym, po mp3) <br>
**Grupa 2** - po 2010 (po serwisami streamingowymi)<br>
$\alpha = 0.5$

In [55]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '2010-01-01']
data1 = data1[data1['data'] >= '2000-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '2010-01-01']
stat, p_value = stats.mannwhitneyu(data1['instrumentalnosc'], data2['instrumentalnosc'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 199417.5
Wartość p-value: 0.1822215346
Brak istotnych różnic między grupami.


#### Wnioski 
Ponownie obserwujemy brak różnic miedzy erą mp3 a serwisami streamingowymi

### Długość utworu a serwisy streamingowe
#### Wybór testu
Powtarzamy krok pierwszy z poprzednich testów.
#### Test Shapiro-Wilka
$H_0$ - dane są opisane rozkładem normalnym <br>
$H_1$ - dane nie są opisane rozkładem normalnym <br>
$\alpha = 0.5$

In [56]:
data = songs_top_10['czas-trwania-m']
stat, p_value = stats.shapiro(data)
print("Statystyka testowa:", stat)
print("Wartość p-value:", p_value)
alpha = 0.05
if p_value > alpha:
    print("Dane wydają się pochodzić z rozkładu normalnego.")
else:
    print("Dane nie wydają się pochodzić z rozkładu normalnego.")

Statystyka testowa: 0.7488442659378052
Wartość p-value: 0.0
Dane nie wydają się pochodzić z rozkładu normalnego.


#### Wybór testu c.d.
Dane ponownie nie pochodzą z rozkłądu normalnego wybieramy zatem test nieparametryczny Mann-Whitneya.
#### Test Mann-Whitneya
**$H_0$** - nie ma istotnej różnicy pomiedzy dwoma populacjiami <br>
**$H_1$** - istnieje istotna różnica pomiedzy dwoma populacjami <br>
**Grupa 1** - 2000 - 2010 (przed serwisami streamingowym, po mp3) <br>
**Grupa 2** - po 2010 (po serwisami streamingowymi)<br>
$\alpha = 0.5$

In [57]:
songs_top_10['data'] = pd.to_datetime(songs_top_10['data'])
data1 = songs_top_10[songs_top_10['data'] < '2010-01-01']
data1 = data1[data1['data'] >= '2000-01-01']
data2 = songs_top_10[songs_top_10['data'] >= '2010-01-01']
stat, p_value = stats.mannwhitneyu(data1['czas-trwania-m'], data2['czas-trwania-m'])
print("Statystyka testowa U:", stat)
print("Wartość p-value:", '%.10f' % p_value)
alpha = 0.05
if p_value > alpha:
    print("Brak istotnych różnic między grupami.")
else:
    print("Obserwowane różnice między grupami są istotne.")

Statystyka testowa U: 256234.5
Wartość p-value: 0.0000000000
Obserwowane różnice między grupami są istotne.


#### Wnioski
Tym razem po wprowadzeniu serwisów streamingowych zaszły różnice pomiedzy długością utworów. Po wprowadzeniu serwisów streamingowych utwory stały się krótsze.
### Wnioski z wpływu serwisów streamingowych
Po wprowadzeniach serwisów streamingowych nie widać dużych zmian w utworach. Jedyną istotną zmiana jest długość utworu. Jednak może on być spowodowany innym czynnikiem.
## Wnioski z testów statystycznych
Pojawienie się nowych instrumentów muzycznych mogło mieć duży wpływ na wyparcie utworów akustycznych przez elektroniczne. Duży wpływ na muzykę na pewno miałą era mp3, która zwiekszyła dostępność do myzyki. Może przez to muzyka jest bardziej liryczna oraz dłużej gości na listach najlpopularniejszych utworów. Co ciekawe wprowadzenie serwisów streamingowych nie miało dużego wpływu na muzykę. Jedyną zmianą jest długość utworów, które stały się krótsze. Jednak może to być spowodowane innym czynnikiem.

## 